## Example code to test saved crystal structure data

In [1]:
import numpy as np
import pickle
import h5py
from tqdm import tqdm
from onsager import crystal, supercell

## Load the hdf5 file with the crystal structure and supercell data

In [2]:
N_units = 5
with h5py.File("CrystData_ortho_{}_cube.h5".format(N_units), "r") as fl:
    lattice = np.array(fl["Lattice_basis_vectors"])
    superlatt = np.array(fl["SuperLatt"])
    basis_cubic = np.array(fl["basis_sites"])
    dxList = np.array(fl["dxList_1nn"])
    NNList = np.array(fl["NNsiteList_sitewise"])
    jumpNewIndices = np.array(fl["JumpSiteIndexPermutation"])
    GroupOpLatticeCartRotMatrices = np.array(fl["GroupOpLatticeCartRotMatrices"])
    GpermNNIdx = np.array(fl["GroupNNPermutation"])

crys = crystal.Crystal(lattice=lattice, basis=[[b for b in basis_cubic]], chemistry=["A"], noreduce=True)
print(crys)
superCell = supercell.ClusterSupercell(crys, superlatt)

x_vac = np.zeros(3)
Rvac, civac = superCell.crys.cart2pos(x_vac)
vacSiteInd = superCell.index(Rvac, civac)[0]
print(vacSiteInd)
jList = NNList[1:, vacSiteInd]

#Lattice:
  a1 = [1. 0. 0.]
  a2 = [0. 1. 0.]
  a3 = [0. 0. 1.]
#Basis:
  (A) 0.0 = [0. 0. 0.]
  (A) 0.1 = [0.  0.5 0.5]
  (A) 0.2 = [0.5 0.  0.5]
  (A) 0.3 = [0.5 0.5 0. ]
0


In [3]:
dxList # contains the jump vectors.

array([[ 0. , -0.5, -0.5],
       [-0. ,  0.5,  0.5],
       [ 0. , -0.5,  0.5],
       [-0. ,  0.5, -0.5],
       [-0.5, -0.5,  0. ],
       [ 0.5,  0.5, -0. ],
       [-0.5,  0. ,  0.5],
       [ 0.5, -0. , -0.5],
       [-0.5,  0. , -0.5],
       [ 0.5, -0. ,  0.5],
       [ 0.5, -0.5,  0. ],
       [-0.5,  0.5, -0. ]])

## Verify Nearest neighbor sites under saved order of jumps

In [4]:
for siteInd in tqdm(range(NNList.shape[1]), position=0, leave=True, ncols=65):
    assert NNList[0, siteInd] == siteInd
    ciSite, Rsite = superCell.ciR(siteInd)
    xSite = superCell.crys.pos2cart(Rsite, ciSite)
    for jmp in range(NNList.shape[0] - 1):
        xSiteNew = xSite + dxList[jmp]
        RsiteNew, ciSiteNew = superCell.crys.cart2pos(xSiteNew)
        siteNew, _ = superCell.index(RsiteNew, ciSiteNew)
        assert NNList[jmp + 1, siteInd] == siteNew

        if siteInd == vacSiteInd:
            assert jList[jmp] == siteNew

100%|█████████████████████████| 500/500 [00:01<00:00, 294.02it/s]


## Now test the group permutation of nearest neigbhors

In [5]:
# We first reconstruct the group operations dictionary
# This will let us test the consistency in the sequence of the group operations
considered = set()
GIndtoGDict = {}
for g in list(superCell.crys.G):
    cartrot = g.cartrot
    for rotInd in range(GroupOpLatticeCartRotMatrices.shape[0]):
        if np.allclose(cartrot, GroupOpLatticeCartRotMatrices[rotInd]):
            assert g not in considered # A group operation cannot be repeated 
            considered.add(g)
            GIndtoGDict[rotInd] = g

In [6]:
# Then test that the nearest neighbors have been consistently stored
for gInd, g in tqdm(GIndtoGDict.items(), position=0, leave=True, ncols=65):
    for jmp in range(dxList.shape[0]):
        jmpvec = dxList[jmp]
        jmpvecRot = superCell.crys.g_cart(g, jmpvec)
        
        idxnew = None
        count = 0
        for jmpNew in range(dxList.shape[0]):
            if np.allclose(dxList[jmpNew], jmpvecRot):
                count += 1
                idxnew = jmpNew
        assert count == 1
        assert GpermNNIdx[gInd, idxnew + 1] == jmp + 1

assert len(GIndtoGDict) == 48

100%|███████████████████████████| 48/48 [00:00<00:00, 120.29it/s]


## Now check re-indexing of sites after jumps, when sites are translated back under PBC so that the vacancy comes back at (0,0,0)

In [7]:
# Now for each jump, displace periodically and check
randomState = np.random.randint(1, 5, NNList.shape[1], dtype=np.int8)
print(randomState.shape)
# Put a vacancy (species 0) at the vacancy site
x_vac = np.zeros(3)
Rvac, civac = superCell.crys.cart2pos(x_vac)
vacSiteInd = superCell.index(Rvac, civac)[0]
randomState[vacSiteInd] = 0
print(vacSiteInd)

for jmp in tqdm(range(dxList.shape[0]), position=0, leave=True, ncols=65):
    
    state2 = randomState.copy()
    assert state2[vacSiteInd] == 0 # check initially there was vacancy
    
    # Then do the vacancy exchange
    assert jList[jmp] == NNList[1+jmp, vacSiteInd]
    state2[vacSiteInd] = state2[jList[jmp]]
    state2[jList[jmp]] = 0
    
    # Now translate the state
    state2Trans = np.zeros_like(state2)
    for site in range(NNList.shape[1]):
        ciSite, Rsite = superCell.ciR(site)
        
        xSite = superCell.crys.pos2cart(Rsite, ciSite)
        
        xSiteNew = xSite - dxList[jmp] # translate by negative of vac jump
        
        RsiteNew, ciSiteNew = superCell.crys.cart2pos(xSiteNew)
        
        assert ciSiteNew is not None
        
        siteIndNew, _ = superCell.index(RsiteNew, ciSiteNew)
        state2Trans[siteIndNew] = state2[site]
    
    assert state2Trans[vacSiteInd] == 0
    assert np.array_equal(randomState[jumpNewIndices[jmp]], state2Trans)

(500,)
0


100%|████████████████████████████| 12/12 [00:01<00:00,  6.97it/s]


## check positions with an ASE supercell to make sure site indices and positions match
Note: For non-diagonal superlatt matrices that break the cubic symmetry of the supercell, the indices do not match between ASE and Onsager. But in all our runs, we'll use diagonal superlattices, for which Onsager and ASE give the same indices.

In [8]:
from ase.spacegroup import crystal as crystal_ASE
from ase.build import make_supercell

In [9]:
a = superCell.crys.lattice[0,0]
print(a)
superlatt = np.diag([N_units, N_units, N_units])

fcc_ASE = crystal_ASE('Ni', [(0, 0, 0)], spacegroup=225, cellpar=[a, a, a, 90, 90, 90], primitive_cell=False)
superFCC_ASE = make_supercell(fcc_ASE, superlatt)

1.0


In [10]:
for site in tqdm(range(len(superFCC_ASE)), position=0, leave=True, ncols=65):
    ciSite, Rsite = superCell.ciR(site)
    x_site = superCell.crys.pos2cart(Rsite, ciSite)
    assert np.allclose(x_site, np.dot(superCell.lattice, superCell.mobilepos[site]), rtol=0, atol=1e-15)
    assert np.allclose(x_site, superFCC_ASE[site].position, atol=1e-15)

100%|████████████████████████| 500/500 [00:00<00:00, 5878.03it/s]
